In [1]:
import numpy as np
import pandas as pd

In [8]:
## constants ##
models_preds_paths = ["./output/scores/val_scores_2-bilstm_ruddit-only.csv", # preds range is [-1, 1]
                     "./output/scores/val_scores_bert-1-BiLstm_2-epoch.csv",# [0, 3/4/5]
                     "./output/scores/val_scores_bert-2-BiLstm_2-epoch.csv",# [0, 3/4/5]
                      "./output/scores/val_scores_original_pretrained_only.csv", # [0, 6] # similar as above
                    "./output/scores/val_scores_unbiased_pretrained_only.csv", # [0, 7]# similar as above
                    "./output/scores/val_scores_multilingual_pretrained_only.csv", # [0, 7] # similar as above
                     ]
              
less_toxic_key = "less_toxic"
more_toxic_key = "more_toxic"

In [3]:
model_preds_df_list = []
for path in models_preds_paths:
    model_preds_df_list.append(pd.read_csv(path))
    print(model_preds_df_list[-1].shape)

(30108, 2)
(30108, 2)
(30108, 2)
(30108, 2)
(30108, 2)
(30108, 2)


In [7]:
val_df = pd.read_csv("./train/validation_data.csv")
val_df, val_df.shape

(       worker                                         less_toxic  \
 0         313            This article sucks \n\nwoo woo wooooooo   
 1         188  "And yes, people should recognize that but the...   
 2          82   Western Media?\n\nYup, because every crime in...   
 3         347  And you removed it! You numbskull! I don't car...   
 4         539   smelly vagina \n\nBluerasberry why don't you ...   
 ...       ...                                                ...   
 30103     461  I'm sorry. I'm not an admin. I will give you t...   
 30104     527  I'm sorry. I'm not an admin. I will give you t...   
 30105     352  wow...\nare you out of your mind, how was my e...   
 30106     311  wow...\nare you out of your mind, how was my e...   
 30107      54  wow...\nare you out of your mind, how was my e...   
 
                                               more_toxic  
 0      WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!...  
 1       Daphne Guinness \n\nTop of the mornin' my

## Bert + 2 BiLSTM Analysis

In [5]:
bert_2_biLSTM = model_preds_df_list[2]
bert_2_biLSTM, bert_2_biLSTM.shape

(       less_toxic  more_toxic
 0        1.143788    0.367601
 1        0.278404    1.492621
 2        0.989652    1.016708
 3        0.843104    3.894335
 4        2.330885    1.460800
 ...           ...         ...
 30103    1.282766    3.454913
 30104    1.282766    3.454913
 30105    0.307757    1.150488
 30106    0.307757    1.150488
 30107    0.307757    1.150488
 
 [30108 rows x 2 columns],
 (30108, 2))

In [11]:
bert_2_bilstm_right_preds = val_df[bert_2_biLSTM[more_toxic_key] > bert_2_biLSTM[less_toxic_key]]
bert_2_bilstm_wrong_preds = val_df[bert_2_biLSTM[more_toxic_key] <= bert_2_biLSTM[less_toxic_key]]
bert_2_bilstm_right_preds.shape, bert_2_bilstm_wrong_preds.shape ## almost 66% accuracy

((20711, 3), (9397, 3))

In [16]:
pd.set_option('display.max_colwidth', None)
bert_2_bilstm_right_preds.head(10)

,worker,less_toxic,more_toxic
1,188,"""And yes, people should recognize that but they usually don't. One of the first objections you hear directed at someone who says he favors free markets is invariably """"There's no such thing as a free market."""" Ridiculously trivial. ""","Daphne Guinness \n\nTop of the mornin' my favourite Alien Life Form. I'm sorry, I can't marry you because you are a poor commoner so, therefore, unimportant. Can I have my dresses back. I hope your fat Oirish arse hasn't stretched them."
2,82,"Western Media?\n\nYup, because every crime in the entire world is caused by the west. Racists.","""Atom you don't believe actual photos of masturbation is sexually explicit photos. I am sorry to say they are sexually explicit and so you said yourself that the disclaimer is used for that purpose. Also, Wikipedia itself is targeted for minors. In addition, you say that Wikipedia does not have child pornography photos. I do believe there is a photo of a real prepubescent child nude (not an illustration) that is in bondage with the title """"Virgin Killer"""". Which recently Wikipedia archived so no more comments could be made. 211.220.23.15 \n\n"""
3,347,"And you removed it! You numbskull! I don't care what you say anymore, this is my life! Go ahead with your own life, leave me alone! —","You seem to have sand in your vagina.\n\nMight want to wash that stinking hole out, bitch."
6,52,"""\n\nHey Dude\nWell, I guess you know that TREY and I got blocked for our """"behaviour"""" when dealing with Kd lvr and Kdkatpir2 and that those two are no longer sockpuppets nor meatpuppets. Which adds to my confusion. Oh well. TREY has quit Wikipedia and I no longer helping, reverting, or reporting anyone. I am going to let them handle it. Hope you have a good day.... (VT) """,gender neutral \n\nwhy is the page called henchmen there are females like Lucy saxon in this page shouldnt it have a gender neutral name like sub-villain
7,672,Also my remarks are indeed a criticism of you and your behavior in relation to the dreamhost talkpage but if you are indeed a NPOV then admin you should perhaps instead of interpreting them as a personal attack read them as just criticism and take steps to improve your conduct.,Why not head on over to WP:RFA and try your hand at getting a mop for all your bullshit? Just think! Maybe one day your useless editing and countless wasted hours/days/weeks here will get you a silly little icon and an excuse to waste even more time. 67.176.98.94
8,52,""" July 2012 (UTC)\n\nI""""M ABUSING PEOPLE!!!!!?????\nYour the one ranting on about how it doesn't exist, the Battle for Australia DID happen it was very similiar to the Battle of Britain, so are you saying the Battle of Britain didn't happen? Of course not, your probably British and will do anything to put down the Aussies, even if it means tampering with our history.\nDon't tell me to read a book on that battle, I know more about it than you do obviously. The Battle for Australia is not historiographcal, ask any veteran about what they thought during the KOKODA CAMPAIGN, and they believed they were """"fighting to save Australia"""". You just hate this country, and your anti-Australian approach shines right through, F*ck you!! 06:41, 4""",You are such a useless bitch! I mean do you have a life or friends? Because all you do is sit at your computer on wikipeida. You are a loser and I would watch out because you will be blocked soon!\n{Talkback|Tdawg4701}
9,539,"It's just simple Beefcake... \n\nThe recent edit I made to Star Trek: Of Gods and Men was constructive, the film really did suck. I mean Bill Shatner's last record was bad but this... Astral please do not abuse Wikipedia and vandalise the article Star Trek: Of Gods and Men again or you will be banned. Beffcake. BEEFCAKE!",Hi \n\nI Hate you. ^_^
10,553,"You are the one who is trolling \n\nI was not trolling. I went to see the user page of this person who kept vandalising my writings in Dog to see what his w

Correct preds maybe bcos of: 
1. toxic/curse words have higher score (expected) 

In [17]:
bert_2_bilstm_wrong_preds.head(10)

,worker,less_toxic,more_toxic
0,313,This article sucks \n\nwoo woo wooooooo,WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!!!!!!!!??????????????????????????????????????????????????????!!!!!!=\nWHER IS YOUR SEXY PIC GONE FROM YOUR MAIN PAGE???????? put it back
4,539,smelly vagina \n\nBluerasberry why don't you be a model for a cheesy blue vagina syndrome. A lot are experiencing this vaginal disease.,"hey \n\nway to support nazis, you racist"
5,437,There's a better answer to the question: the people that made the anime fucked up.,I'm not new here. I don't know what the hell you are talking about. Don't talk to me again.
15,540,page. Untill then don't waste my time by posting crap things on my,"! \n\nShut your mouth! Dont tell me to be civil because I am done here. I will not log back into this site again or contribute to it again. Everybody here wants to act like a jackass thats fine, because I dont have to tolerate it."
16,258,"""\nI think you went overboard here and blew your load a little too quickly by blocking me for a week for posting on my own talk page. It really is quite ridiculous. I have understood that users on wikipedia have far greater leeway on their own user talk pages and user pages, and asking other users to stay off is an accepted practice. My comments on the Sarah Palin edit summary probably went over the line and deserved either a warning or 1 day block because of the article probation, and I should not have reacted the way I did to the other users instigation. But again I note that I did not notice you posting a similar warning to the user who said in his edit summary towards me """"then stop posting biased crap"""" . \nYour entitled to do whatever the fuck you want on wikipedia Coffee because you have the powers which are very hard to take away, but really learn how to read what you link to. Lookup what the definition of harassment, I didn't threaten or try to intimidate you. I might have insulted you and your fragile sensibilities, but I certainly did not harass you. I stick by what I said though, please fuck off my talk page after this situation is done. Keep fucking that chicken. Bits andPieces ""","User Talk \n\nYOU ARE VANDALIZING MY PAGE. PLEASE HALT. I tried to resolve this peacefully and without incident, howevever you are violating my rights as a user. I am trying to compromise and you aren't interested in the least in listening. Let us discuss this FOR THE FINAL TIME."
21,277,Pontiac Montana \n\nIf someone ever adds a sorce that says it wont be discounted i will laugh in your face karrmann because ibelive them more than you you are full of shit\n\nleave this article alone or i will report you \n\nF,"""\n\nI think the key words here are """"claim to have been molested."""" Justin was an Internet sex entrepreneur and a high paid male prostitute, who had numerous opportunities to get out of the business had he wanted to do so. It is only society's blind and blanket characterization of everyone under the age of 18 as having been """"molested"""" when they engage in problematical sexual activity, which permits Justin to make a quick transition from selling his body to selling his victimhood. Rookiee's article was fact-based and properly sourced. That Justin didn't like it when the facts weren't spun from a victimologist perspective is immaterial.\n\nWikipedia pretends that it's run by consensus as long as volunteers are happily creating multiple millions of dollars in free intellectual property for Jimbo. Once that task is largely complete, the truth is revealed, which is that this is basically Jimbo's private BBS where he can do anything he feels like, and the contributions he paid nothing for are his to exploit any way he wishes.\n\nGood luck with your Oprah-esque fluff piece on the attention-seeking camwhore. \n\n"""
22,22,"LGagnon \ni for one, think lgagnon is a big fat cock sucker. lgagnon? Lgagnon my nuts.","Patrick J. Kennedy\nWhy did you delete my edit to the Patrick Kennedy article? My edit wa

In [39]:
bert_2_biLSTM.loc[32]

less_toxic    0.234526
more_toxic    0.242691
Name: 32, dtype: float64

Predicting wrong where:
1. index 4: model is scoring higher on curse words than identity hate -> according to annotators identity hate (nazis) should rank higher;. ie having curse automatically makes it higher. Almost double the weight. 
2. index 0 ** -> curse word sucks is ranked higher than non curse sentence which are more offensive. according to annotators sexual harassment is ranked higher
3. index 5: similar reason
4. index 15: both has curse so close score. (1.681268, 1.450271)  second has civil in it (maybe bit less score?)
5. index 16**: interesting case. both are close case, according to me but model ranks 20 times just bcos of fuck. Think CAPITAL letters matters


# seems a simple model

## Unitary AI

In [26]:
original = model_preds_df_list[3]
original, original.shape

(       less_toxic  more_toxic
 0        2.157796    1.507812
 1        0.002027    1.909370
 2        0.080560    0.372039
 3        1.370524    3.252640
 4        2.605629    2.015862
 ...           ...         ...
 30103    0.591702    2.466029
 30104    0.591702    2.466029
 30105    0.306528    2.357453
 30106    0.306528    2.357453
 30107    0.306528    2.357453
 
 [30108 rows x 2 columns],
 (30108, 2))

In [27]:
original_right_preds = val_df[original[more_toxic_key] > original[less_toxic_key]]
original_wrong_preds = val_df[original[more_toxic_key] <= original[less_toxic_key]]
original_right_preds.shape, original_wrong_preds.shape ## almost 66% accuracy

((20875, 3), (9233, 3))

In [30]:
original_right_preds.head(10)

,worker,less_toxic,more_toxic
1,188,"""And yes, people should recognize that but they usually don't. One of the first objections you hear directed at someone who says he favors free markets is invariably """"There's no such thing as a free market."""" Ridiculously trivial. ""","Daphne Guinness \n\nTop of the mornin' my favourite Alien Life Form. I'm sorry, I can't marry you because you are a poor commoner so, therefore, unimportant. Can I have my dresses back. I hope your fat Oirish arse hasn't stretched them."
2,82,"Western Media?\n\nYup, because every crime in the entire world is caused by the west. Racists.","""Atom you don't believe actual photos of masturbation is sexually explicit photos. I am sorry to say they are sexually explicit and so you said yourself that the disclaimer is used for that purpose. Also, Wikipedia itself is targeted for minors. In addition, you say that Wikipedia does not have child pornography photos. I do believe there is a photo of a real prepubescent child nude (not an illustration) that is in bondage with the title """"Virgin Killer"""". Which recently Wikipedia archived so no more comments could be made. 211.220.23.15 \n\n"""
3,347,"And you removed it! You numbskull! I don't care what you say anymore, this is my life! Go ahead with your own life, leave me alone! —","You seem to have sand in your vagina.\n\nMight want to wash that stinking hole out, bitch."
6,52,"""\n\nHey Dude\nWell, I guess you know that TREY and I got blocked for our """"behaviour"""" when dealing with Kd lvr and Kdkatpir2 and that those two are no longer sockpuppets nor meatpuppets. Which adds to my confusion. Oh well. TREY has quit Wikipedia and I no longer helping, reverting, or reporting anyone. I am going to let them handle it. Hope you have a good day.... (VT) """,gender neutral \n\nwhy is the page called henchmen there are females like Lucy saxon in this page shouldnt it have a gender neutral name like sub-villain
7,672,Also my remarks are indeed a criticism of you and your behavior in relation to the dreamhost talkpage but if you are indeed a NPOV then admin you should perhaps instead of interpreting them as a personal attack read them as just criticism and take steps to improve your conduct.,Why not head on over to WP:RFA and try your hand at getting a mop for all your bullshit? Just think! Maybe one day your useless editing and countless wasted hours/days/weeks here will get you a silly little icon and an excuse to waste even more time. 67.176.98.94
8,52,""" July 2012 (UTC)\n\nI""""M ABUSING PEOPLE!!!!!?????\nYour the one ranting on about how it doesn't exist, the Battle for Australia DID happen it was very similiar to the Battle of Britain, so are you saying the Battle of Britain didn't happen? Of course not, your probably British and will do anything to put down the Aussies, even if it means tampering with our history.\nDon't tell me to read a book on that battle, I know more about it than you do obviously. The Battle for Australia is not historiographcal, ask any veteran about what they thought during the KOKODA CAMPAIGN, and they believed they were """"fighting to save Australia"""". You just hate this country, and your anti-Australian approach shines right through, F*ck you!! 06:41, 4""",You are such a useless bitch! I mean do you have a life or friends? Because all you do is sit at your computer on wikipeida. You are a loser and I would watch out because you will be blocked soon!\n{Talkback|Tdawg4701}
9,539,"It's just simple Beefcake... \n\nThe recent edit I made to Star Trek: Of Gods and Men was constructive, the film really did suck. I mean Bill Shatner's last record was bad but this... Astral please do not abuse Wikipedia and vandalise the article Star Trek: Of Gods and Men again or you will be banned. Beffcake. BEEFCAKE!",Hi \n\nI Hate you. ^_^
10,553,"You are the one who is trolling \n\nI was not trolling. I went to see the user page of this person who kept vandalising my writings in Dog to see what his w

Good thing:
1. model ranks curse higher 

In [29]:
original_wrong_preds.head(10)

,worker,less_toxic,more_toxic
0,313,This article sucks \n\nwoo woo wooooooo,WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!!!!!!!!??????????????????????????????????????????????????????!!!!!!=\nWHER IS YOUR SEXY PIC GONE FROM YOUR MAIN PAGE???????? put it back
4,539,smelly vagina \n\nBluerasberry why don't you be a model for a cheesy blue vagina syndrome. A lot are experiencing this vaginal disease.,"hey \n\nway to support nazis, you racist"
5,437,There's a better answer to the question: the people that made the anime fucked up.,I'm not new here. I don't know what the hell you are talking about. Don't talk to me again.
16,258,"""\nI think you went overboard here and blew your load a little too quickly by blocking me for a week for posting on my own talk page. It really is quite ridiculous. I have understood that users on wikipedia have far greater leeway on their own user talk pages and user pages, and asking other users to stay off is an accepted practice. My comments on the Sarah Palin edit summary probably went over the line and deserved either a warning or 1 day block because of the article probation, and I should not have reacted the way I did to the other users instigation. But again I note that I did not notice you posting a similar warning to the user who said in his edit summary towards me """"then stop posting biased crap"""" . \nYour entitled to do whatever the fuck you want on wikipedia Coffee because you have the powers which are very hard to take away, but really learn how to read what you link to. Lookup what the definition of harassment, I didn't threaten or try to intimidate you. I might have insulted you and your fragile sensibilities, but I certainly did not harass you. I stick by what I said though, please fuck off my talk page after this situation is done. Keep fucking that chicken. Bits andPieces ""","User Talk \n\nYOU ARE VANDALIZING MY PAGE. PLEASE HALT. I tried to resolve this peacefully and without incident, howevever you are violating my rights as a user. I am trying to compromise and you aren't interested in the least in listening. Let us discuss this FOR THE FINAL TIME."
17,104,free me \n\ni have been banned free me you are my only hope of freedom from being banned,"This has got to be the 10th time I've seen someone with this same complaint. The fact of the matter is that Wikipedia's editors are primarily young white males, and what do young white males enjoy more than pop culture? 98.207.43.214"
21,277,Pontiac Montana \n\nIf someone ever adds a sorce that says it wont be discounted i will laugh in your face karrmann because ibelive them more than you you are full of shit\n\nleave this article alone or i will report you \n\nF,"""\n\nI think the key words here are """"claim to have been molested."""" Justin was an Internet sex entrepreneur and a high paid male prostitute, who had numerous opportunities to get out of the business had he wanted to do so. It is only society's blind and blanket characterization of everyone under the age of 18 as having been """"molested"""" when they engage in problematical sexual activity, which permits Justin to make a quick transition from selling his body to selling his victimhood. Rookiee's article was fact-based and properly sourced. That Justin didn't like it when the facts weren't spun from a victimologist perspective is immaterial.\n\nWikipedia pretends that it's run by consensus as long as volunteers are happily creating multiple millions of dollars in free intellectual property for Jimbo. Once that task is largely complete, the truth is revealed, which is that this is basically Jimbo's private BBS where he can do anything he feels like, and the contributions he paid nothing for are his to exploit any way he wishes.\n\nGood luck with your Oprah-esque fluff piece on the attention-seeking camwhore. \n\n"""
22,22,"LGagnon \ni for one, think lgagnon is a big fat cock sucker. lgagnon? Lgagnon my nuts.","Patrick J. Kennedy\nWhy did you delete my edit to the Patrick Ke

In [42]:
original.loc[0]

less_toxic    2.157796
more_toxic    1.507812
Name: 0, dtype: float64

bad thing:
1. index 0: sexual harassment should rank higher according to annotator. looks like curse word got ranked higher. Good thing, scores are not so far apart (but still significant) compared to Bert 
2. index 4 -> similar. scores even closer. racist stuff(nazis) should rank higher than curse word (vaginal) acc. to anno.
3. index 17** -> model could not rank racism (in terms of white male non offensive language ) higher. Good thing score is too close. Possible wrong bcos of word ban?
4. index 22 ** -> model score are very apart (100 times!). straight up abuse language  should rank less than homosex. stuff acc. to  anno. model ranked curse words way higher. can be one off (for me as well. looks like anno. bias. but scores should been close. kind of has scarcasm as well). All model might go wrong here
5. index 27 -> scores are too apart (20 times). tough one.
6. index 32 -> 10 times apart. Bert predicted correctly. hmmm
7. index 37** -> 20 times apart are the scores. Offensive language gave higher score. interesting one. something about bad guy definitely should rank lower than supposed lying politician (makes sense in a non contextual manner). model seems to be missing context in general. bad guy -> offensive lang may be acceptable. Tough one

# model has problem with non offensive language to predict correctly! Must be similar with unbiased and multingual as well.